### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [16]:
movie_mat = np.random.rand(4,4)
print(movie_mat)
for i in movie_mat:
    for j in i:
        print("\n", j)

[[0.40971363 0.84166788 0.28170935 0.40218341]
 [0.8573548  0.14821751 0.12213245 0.09728095]
 [0.333948   0.90675857 0.24353923 0.32211613]
 [0.28379629 0.19687968 0.28949398 0.7584672 ]]

 0.4097136314649835

 0.8416678776385681

 0.28170935368927486

 0.40218341152206416

 0.85735479811684

 0.14821751498862268

 0.12213244626874942

 0.09728095265030912

 0.3339479958755216

 0.9067585730446895

 0.2435392305743609

 0.3221161259432489

 0.2837962851511634

 0.19687968171932002

 0.28949398016237027

 0.7584671979497223


In [26]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.sum(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features )# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for h in range(n_movies):
                # if the rating exists
                if ratings_mat[i,h] > 0:
        
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[i,h] - np.dot(user_mat[i, :], movie_mat[:, h])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error**2

                    # update the values in each matrix in the direction of the gradient
                    for x in range(latent_features):
                        user_mat[i,x] += learning_rate * (2 * error * movie_mat[x,h])
                        movie_mat[x,h] += learning_rate * (2 * error * user_mat[i,x])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))

        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [27]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 48.381282
2 		 19.545089
3 		 5.271178
4 		 2.963819
5 		 2.753546
6 		 2.717205
7 		 2.698137
8 		 2.680323
9 		 2.661041
10 		 2.639537


In [28]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.20607933  8.7712631  10.71098085 10.04786673]
 [ 8.65411952  7.35552828  9.07337173  8.84887124]
 [ 7.98000267  7.07794407  8.28532959  7.55204541]
 [ 9.6383224   8.16541504 10.09028108  9.55546534]
 [ 8.65211783  7.43141606  9.09012414  8.36894714]
 [ 6.90708474  5.98776641  7.23148284  6.8040953 ]
 [ 9.30715681  7.95171082  9.72911113  9.14239498]
 [ 8.33248923  6.96809685  8.69041283  8.17527873]
 [ 8.48638591  7.38416192  8.92262186  8.49179801]
 [ 7.60322901  6.69176211  7.9258101   7.8505879 ]
 [ 8.91136895  7.80402257  9.34741491  8.7992733 ]
 [ 9.5997582   8.7292177   9.96364125  9.45668357]
 [ 9.18924381  8.05881762  9.64396754  9.39807471]
 [ 6.6226505   5.85025591  6.94621146  6.60404612]
 [ 9.78433236  8.42286517 10.21896979  9.7813427 ]
 [ 9.5964      8.52046983 10.01269506 10.02558225]
 [ 8.30286704  7.55859461  8.65686672  8.14971737]
 [ 7.05504571  6.19228319  7.34302776  7.21673651]
 [ 8.30465232  7.33970442  8.70806837  8.51371422]
 [ 8.15736395  7.09027557  8.60

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [30]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

126 		 0.000018
127 		 0.000016
128 		 0.000014
129 		 0.000013
130 		 0.000011
131 		 0.000010
132 		 0.000009
133 		 0.000008
134 		 0.000007
135 		 0.000006
136 		 0.000006
137 		 0.000005
138 		 0.000005
139 		 0.000004
140 		 0.000004
141 		 0.000003
142 		 0.000003
143 		 0.000003
144 		 0.000002
145 		 0.000002
146 		 0.000002
147 		 0.000002
148 		 0.000001
149 		 0.000001
150 		 0.000001
151 		 0.000001
152 		 0.000001
153 		 0.000001
154 		 0.000001
155 		 0.000001
156 		 0.000001
157 		 0.000001
158 		 0.000000
159 		 0.000000
160 		 0.000000
161 		 0.000000
162 		 0.000000
163 		 0.000000
164 		 0.000000
165 		 0.000000
166 		 0.000000
167 		 0.000000
168 		 0.000000
169 		 0.000000
170 		 0.000000
171 		 0.000000
172 		 0.000000
173 		 0.000000
174 		 0.000000
175 		 0.000000
176 		 0.000000
177 		 0.000000
178 		 0.000000
179 		 0.000000
180 		 0.000000
181 		 0.000000
182 		 0.000000
183 		 0.000000
184 		 0.000000
185 		 0.000000
186 		 0.000000
187 		 0.000000
188 		 0

In [31]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.00000329 10.00000177 10.00000074  9.99999657]
 [ 9.999999    3.9999994   8.99999973 10.00000105]
 [ 8.00000092  9.00000042 10.00000014  4.99999905]
 [ 9.00000418  8.00000221 10.00000089  9.99999562]
 [10.00000037  5.00000002  8.99999996  8.99999967]
 [ 5.99999702  3.99999832  9.99999924  6.00000309]
 [ 8.99999517  7.99999712  9.99999875  9.00000519]
 [10.00000026  5.00000024  9.00000012  7.99999966]
 [ 7.000003    8.00000159 10.00000066  7.99999689]
 [ 9.00000495  5.00000273  9.00000113  6.99999476]
 [ 8.99998865  7.99999364  9.99999726  8.00001194]
 [ 9.00000277 10.00000163 10.00000072  8.99999707]
 [ 9.99999487  8.99999727  9.99999886  8.00000535]
 [ 4.99999808  7.99999894  4.99999957  8.00000207]
 [10.00000214  8.0000013  10.00000063  9.99999777]
 [ 8.99999577  8.99999766  9.99999907 10.00000455]
 [ 9.00000568  8.00000331  8.00000147  7.999994  ]
 [ 9.99999893  7.9999994   0.99999974 10.00000113]
 [ 5.00000031  6.00000029 10.00000015  9.99999965]
 [ 8.00000421  7.00000234 10.00

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [32]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [34]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

126 		 0.000017
127 		 0.000015
128 		 0.000013
129 		 0.000012
130 		 0.000010
131 		 0.000009
132 		 0.000008
133 		 0.000007
134 		 0.000007
135 		 0.000006
136 		 0.000005
137 		 0.000005
138 		 0.000004
139 		 0.000004
140 		 0.000003
141 		 0.000003
142 		 0.000003
143 		 0.000002
144 		 0.000002
145 		 0.000002
146 		 0.000002
147 		 0.000001
148 		 0.000001
149 		 0.000001
150 		 0.000001
151 		 0.000001
152 		 0.000001
153 		 0.000001
154 		 0.000001
155 		 0.000001
156 		 0.000001
157 		 0.000000
158 		 0.000000
159 		 0.000000
160 		 0.000000
161 		 0.000000
162 		 0.000000
163 		 0.000000
164 		 0.000000
165 		 0.000000
166 		 0.000000
167 		 0.000000
168 		 0.000000
169 		 0.000000
170 		 0.000000
171 		 0.000000
172 		 0.000000
173 		 0.000000
174 		 0.000000
175 		 0.000000
176 		 0.000000
177 		 0.000000
178 		 0.000000
179 		 0.000000
180 		 0.000000
181 		 0.000000
182 		 0.000000
183 		 0.000000
184 		 0.000000
185 		 0.000000
186 		 0.000000
187 		 0.000000
188 		 0

In [38]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.75828312609834:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [39]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.510507
2 		 10.875398
3 		 7.421955
4 		 5.711398
5 		 4.638255
6 		 3.888980
7 		 3.331989
8 		 2.900858
9 		 2.557676
10 		 2.278813
11 		 2.048615
12 		 1.856254
13 		 1.693977
14 		 1.556047
15 		 1.438089
16 		 1.336661
17 		 1.248991
18 		 1.172811
19 		 1.106247
20 		 1.047751


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [40]:
preds = np.dot(user_mat, movie_mat)

In [41]:
# Replace each of the comments below with the correct values
num_ratings = np.sum(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.sum(~np.isnan(preds)) - num_ratings # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [42]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
